In [17]:
import numpy as np
import nibabel as nib
from nibabel.affines import apply_affine
from nibabel.streamlines import Tractogram, TckFile 
from fury import window, actor
from tqdm import tqdm  # Opcional: para barra de progreso


def cargar_datos(eigenvector_path, mask_path):
    """
    Carga los datos de eigenvectores y de la máscara binaria.
    
    Parámetros:
      eigenvector_path: Ruta al archivo de eigenvectores (NIfTI).
      mask_path: Ruta al archivo de la máscara binaria (NIfTI).
      
    Retorna:
      eigen_data: Arreglo numpy de eigenvectores con forma (X,Y,Z,3).
      mask_data: Arreglo numpy binario con forma (X,Y,Z).
      affine: Matriz de transformación del espacio imagen.
    """
    eigen_img = nib.load(eigenvector_path)
    eigen_data = eigen_img.get_fdata()
    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    affine = eigen_img.affine  # se asume que ambos archivos comparten la misma transformación

    # En cargar_datos, verifica la consistencia:
    affine_eigen = eigen_img.affine
    affine_mask = mask_img.affine
    if not np.allclose(affine_eigen, affine_mask):
        print("¡Advertencia! Las matrices afines de la imagen de eigenvectores y la máscara son diferentes.")
        # Decide cuál usar, por ejemplo, la de los eigenvectores:
        affine = affine_eigen
        # Aquí podrías añadir lógica para transformar la máscara si fuera necesario,
        # aunque es mejor hacerlo como un paso de preprocesamiento separado.
    else:
        print("Matrices afines iguales.")
        affine = affine_eigen # Son iguales, usa cualquiera
    
    # print(eigen_data.shape)
    # print(mask_data.shape)
    # print(eigen_data)
    # print(mask_data)
    return eigen_data, mask_data, affine
    
    
def tracking_streamline_bidireccional(eigen_data, mask, affine, seed, step_size=1.0, max_steps=1000):
    """
    Reconstruye un streamline a partir de una semilla en ambas direcciones usando tractografía determinista.
    
    Parámetros:
      eigen_data: Arreglo (X, Y, Z, 3, 3) con los eigenvectores ordenados por importancia.
      mask: Arreglo binario (X, Y, Z) que indica regiones válidas para tracking.
      affine: Matriz de transformación de coordenadas voxel a mundo real.
      seed: Coordenadas en espacio real del punto semilla (3,).
      step_size: Tamaño del paso en unidades espaciales.
      max_steps: Número máximo de iteraciones por dirección.
      
    Retorna:
      streamline: Arreglo numpy (N, 3) de puntos en espacio real que conforman el streamline.
    """
    inv_affine = np.linalg.inv(affine)

    def track(direction_sign=1):
        points = []
        current_point = np.array(seed)
        for _ in range(max_steps):
            # Transformar al espacio voxel
            voxel_coords = np.round(apply_affine(inv_affine, current_point)).astype(int)

            # Validar que está dentro del volumen y la máscara
            if np.any(voxel_coords < 0) or np.any(voxel_coords >= mask.shape):
                break
            if not mask[tuple(voxel_coords)]:
                break

            # Extraer el eigenvector principal (primera columna del array 3x3)
            vec = eigen_data[voxel_coords[0], voxel_coords[1], voxel_coords[2], :, 0]
            norm_vec = np.linalg.norm(vec)
            if norm_vec == 0:
                break
            direction_voxel = direction_sign * (vec / norm_vec)

            # Transformar dirección a espacio real
            direction_real = affine[:3, :3] @ direction_voxel
            norm_real = np.linalg.norm(direction_real)
            if norm_real == 0:
                break
            direction = direction_real / norm_real

            # Actualizar punto
            new_point = current_point + step_size * direction

            # Verificar que el nuevo punto también esté en una región válida
            voxel_new = np.round(apply_affine(inv_affine, new_point)).astype(int)
            if np.any(voxel_new < 0) or np.any(voxel_new >= mask.shape):
                break
            if not mask[tuple(voxel_new)]:
                break

            points.append(new_point)
            current_point = new_point
        return points

    # Tracking en ambas direcciones
    forward = [np.array(seed)] + track(direction_sign=+1)
    backward = track(direction_sign=-1)
    backward.reverse()  # Para que el sentido sea correcto

    streamline = backward + forward
    return np.vstack(streamline)


def generate_multiple_streamlines_tck(
    n_streamlines_target,
    seed_points_list,
    output_tck_filename,
    eigen_data,
    mask,
    affine,
    step_size=1.0,
    max_steps=1000,
    verbose=True
):
    generated_streamlines = []
    covered_voxels = set()

    try:
        inv_affine = np.linalg.inv(affine)
    except np.linalg.LinAlgError:
        print("Error: La matriz afín no es invertible.")
        return []

    # Vóxeles válidos para semillado automático
    valid_voxel_indices = np.argwhere(mask)
    rng = np.random.default_rng(seed=42)  # Control de aleatoriedad para reproducibilidad

    if verbose:
        print(f"Intentando generar {n_streamlines_target} streamlines...")
        pbar = tqdm(total=n_streamlines_target, desc="Generando Streamlines")

    # Iterador inicial
    seed_iterator = iter(seed_points_list)
    seeds_processed = 0
    total_seeds_generated = len(seed_points_list)

    while len(generated_streamlines) < n_streamlines_target:
        try:
            current_seed_mm = next(seed_iterator)
        except StopIteration:
            # Agotar la lista original: generar nueva semilla aleatoria en región válida
            while True:
                voxel_idx = tuple(valid_voxel_indices[rng.integers(len(valid_voxel_indices))])
                if voxel_idx not in covered_voxels:
                    break  # Encontramos un vóxel no cubierto
            current_seed_mm = apply_affine(affine, voxel_idx)
            total_seeds_generated += 1

        seeds_processed += 1
        seed_voxel_float = apply_affine(inv_affine, current_seed_mm)
        seed_voxel_int = tuple(np.round(seed_voxel_float).astype(int))

        if seed_voxel_int in covered_voxels:
            continue

        new_streamline_mm = tracking_streamline_bidireccional(
            eigen_data, mask, affine, current_seed_mm, step_size, max_steps
        )

        if new_streamline_mm is not None and new_streamline_mm.shape[0] > 1:
            generated_streamlines.append(new_streamline_mm)
            pbar.update(1)

            streamline_voxels_float = apply_affine(inv_affine, new_streamline_mm)
            streamline_voxels_int = np.round(streamline_voxels_float).astype(int)
            unique_voxels_in_streamline = set(map(tuple, streamline_voxels_int))
            covered_voxels.update(unique_voxels_in_streamline)
        elif verbose and seeds_processed % 50 == 0:
            # print(f"\nFallo al generar streamline desde semilla {seeds_processed} (vóxel {seed_voxel_int}).")

    if verbose:
        pbar.close()
        print(f"\nGeneración completada. Se generaron {len(generated_streamlines)} streamlines.")
        print(f"Se procesaron {seeds_processed} puntos semilla (incluyendo auto-generadas).")

    if generated_streamlines:
        header = {
            nib.streamlines.Field.VOXEL_TO_RASMM: affine,
            nib.streamlines.Field.VOXEL_SIZES: nib.affines.voxel_sizes(affine),
            nib.streamlines.Field.DIMENSIONS: mask.shape,
        }

        tractogram = nib.streamlines.Tractogram(generated_streamlines, affine_to_rasmm=affine)
        tck_obj = nib.streamlines.tck.TckFile(tractogram, header=header)

        try:
            nib.streamlines.save(tck_obj, output_tck_filename)
            if verbose:
                print(f"Streamlines guardados en: {output_tck_filename}")
        except Exception as e:
            print(f"Error al guardar el archivo .tck: {e}")
    elif verbose:
        print("No se generaron streamlines válidos para guardar.")

    return generated_streamlines

def guardar_tck(streamline, affine, output_file):
    """
    Guarda el streamline en un archivo .tck.
    
    Parámetros:
      streamline: Arreglo numpy con la trayectoria (N x 3).
      affine: Matriz de transformación (voxel a RASmm).
      output_file: Ruta del archivo de salida .trk.
    """
    # Se crea un tractograma con el streamline
    tractogram = Tractogram([streamline], affine_to_rasmm=affine)
    # trk_obj = TrkFile(tractogram, header={})
    tck_obj = TckFile(tractogram, header={})
    # nib.streamlines.save(trk_obj, output_file)
    nib.streamlines.save(tck_obj, output_file)
    print(f"Streamline guardado como: {output_file}")

def visualizar_streamline(streamline):
    """
    Visualiza en 3D el streamline utilizando fury.
    
    Parámetros:
      streamline: Arreglo numpy de puntos del streamline.
    """
    scene = window.Scene()
    # Se crea un actor de línea (recibe una lista de streamlines, cada uno como un array Nx3)
    line_actor = actor.line([streamline], colors=(1, 0, 0))
    scene.add(line_actor)
    window.show(scene)

In [27]:
# inputs
eigenvector_file = 'tensor_evecs_stanford.nii.gz'
mask_file = 'mask_stanford.nii.gz'                
seed_point = [40, 5, 4]                 
    
eigen_data, mask_data, affine = cargar_datos(eigenvector_file, mask_file)
    
# Reconstrucción del streamline 
streamline = tracking_streamline_bidireccional(eigen_data, mask_data, affine, seed_point, step_size=0.5, max_steps=2000)
    

output_trk = 'streamline_bidirec_stanford.tck' # cambiar a .tck
guardar_tck(streamline, affine, output_trk)
    
visualizar_streamline(streamline)


Streamline guardado como: streamline_bidirec_stanford.tck


In [20]:
if __name__ == '__main__':

    print("Ejecutando ejemplo...")

    # Carga datos eigen, mask y affine 
    # mask_img = nib.load('mask_stanford.nii.gz')
    # mask_data = mask_img.get_fdata().astype(bool) 
    # affine_matrix = mask_img.affine
    # eigen_img = nib.load('tensor_evecs_stanford.nii.gz')
    # eigen_vectors = eigen_img.get_fdata()
    # img_shape = mask_data.shape
    eigen_vectors, mask_data, affine_matrix = cargar_datos('tensor_evecs_stanford.nii.gz','mask_stanford.nii.gz')
    img_shape = mask_data.shape
    print(eigen_vectors.shape, img_shape)

    # Lista de puntos semilla en mm (coordenadas del mundo real)
    # Definimos el centro y tamaño del cubo en mm
    center_mm = nib.affines.apply_affine(affine_matrix, np.array(img_shape)/2.0 - 0.5 ) # Centro de la imagen en mm
    box_size_mm = 10.0
    num_seeds = 5000
    seed_list_mm = [center_mm + (np.random.rand(3) - 0.5) * box_size_mm for _ in range(num_seeds)]

    # Num de streamlines
    target_count = 5000
    output_file = f'tractograma_multiple_{box_size_mm}_{num_seeds}_{target_count}.tck'

    # Genera tractrograma
    generated_streamlines_list = generate_multiple_streamlines_tck(
        n_streamlines_target=target_count,
        seed_points_list=seed_list_mm,
        output_tck_filename=output_file,
        eigen_data=eigen_vectors, 
        mask=mask_data,             
        affine=affine_matrix,         
        step_size=0.1,                   
        max_steps=1000,                   
        verbose=True
    )

    print(f"\nEjemplo finalizado. Se generaron {len(generated_streamlines_list)} streamlines.")

Ejecutando ejemplo...
Matrices afines iguales.
(71, 88, 62, 3, 3) (71, 88, 62)
Intentando generar 5000 streamlines...


Generando Streamlines:   1%|          | 32/5000 [00:02<05:04, 16.31it/s]


Fallo al generar streamline desde semilla 150 (vóxel (np.int64(37), np.int64(45), np.int64(30))).


Generando Streamlines:   1%|          | 36/5000 [00:02<05:20, 15.48it/s]


Fallo al generar streamline desde semilla 500 (vóxel (np.int64(36), np.int64(45), np.int64(29))).

Fallo al generar streamline desde semilla 700 (vóxel (np.int64(35), np.int64(43), np.int64(29))).

Fallo al generar streamline desde semilla 1000 (vóxel (np.int64(37), np.int64(43), np.int64(29))).

Fallo al generar streamline desde semilla 1150 (vóxel (np.int64(36), np.int64(43), np.int64(29))).

Fallo al generar streamline desde semilla 1200 (vóxel (np.int64(36), np.int64(43), np.int64(28))).

Fallo al generar streamline desde semilla 1450 (vóxel (np.int64(37), np.int64(44), np.int64(29))).

Fallo al generar streamline desde semilla 1550 (vóxel (np.int64(35), np.int64(43), np.int64(29))).

Fallo al generar streamline desde semilla 1750 (vóxel (np.int64(37), np.int64(42), np.int64(29))).

Fallo al generar streamline desde semilla 2050 (vóxel (np.int64(35), np.int64(44), np.int64(29))).

Fallo al generar streamline desde semilla 2200 (vóxel (np.int64(37), np.int64(43), np.int64(29))).

F

Generando Streamlines: 100%|██████████| 5000/5000 [07:57<00:00, 10.48it/s]



Generación completada. Se generaron 5000 streamlines.
Se procesaron 9963 puntos semilla (incluyendo auto-generadas).
Streamlines guardados en: tractograma_multiple_10.0_5000_5000.tck

Ejemplo finalizado. Se generaron 5000 streamlines.


- Calcular todo en el espacio de la imagen y convertir hasta el final al espacio real
- Añadir la función del vecindario con dirección promediada
- verificar si la direccion actual es paralela y no antiparalela a la nueva
- script para generacion de tuplas de campo - direccion
- modelo MLP, o CNN con LSTM